**British Airways Revenue Forecasting 2** <br>
Oxford Saïd Business School <br>
Analysis 2: Create Graphics <br>
<br>
*Jake Schneider* <br>
MBA Candidate <br>
November 2020

----

**Load Libraries**

In [1]:
#Import libraries
import sys
import pandas as pd
from datetime import date, datetime, time, timedelta
import pendulum
import json
import requests
import numpy as np
from numpy import NaN
from numpy import nanmean
import math

import collections

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='ticks', context='talk')

from matplotlib.offsetbox import AnchoredText
from sklearn import linear_model
from sklearn.metrics import r2_score
import statsmodels.api as sm
#import fbprophet

import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import datapane as dp

import warnings
import itertools

----

**Load Clean BA Data Set**

In [2]:
# Load CFA Data from CFA_analysis_1

df = pd.read_csv('../3_Outputs/Data/ba_df.csv')

In [3]:
# Check Types

#print(df.dtypes)

In [4]:
# Confirm Types

print(df['Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units'].dtypes)
df['Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units'] = df['Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units'].astype('float64')
print(df['Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units'].dtypes)

float64
float64


In [5]:
# Confirm Types

print(df['region'].dtypes)
df['region'].astype('string')
print(df['region'].dtypes)

object
object


In [6]:
# View DF

df

,id,iso2Code,country,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude,...,"Inflation, end of period consumer prices, Percent change,","Output gap in percent of potential GDP, Percent of potential GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,","Volume of Imports of goods, Percent change,","Volume of exports of goods and services, Percent change,","Volume of exports of goods, Percent change,","Volume of imports of goods and services, Percent change,"
0,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9978,ZWE,ZW,Zimbabwe,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,Blend,Harare,31.0672,-17.8312,...,3,NaN,15.49,NaN,NaN,NaN,29.6,NaN,6.11,NaN
9979,ZWE,ZW,Zimbabwe,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,Blend,Harare,31.0672,-17.8312,...,3,NaN,15.82,NaN,NaN,NaN,30.92,NaN,7.55,NaN
9980,ZWE,ZW,Zimbabwe,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,Blend,Harare,31.0672,-17.8312,...,3,NaN,16.16,NaN,NaN,NaN,32.17,NaN,5.95,NaN
9981,ZWE,ZW,Zimbabwe,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,Blend,Harare,31.0672,-17.8312,...,3,NaN,16.48,NaN,NaN,NaN,33.81,NaN,5.83,NaN


-----

**Data Analysis & Cleaning**

*Group By Average Value by Continent*

In [87]:
## Create filter to sum total_population
#
#sum_cols = ['binary_val']
#agg_dict = {col: 'sum' if col in sum_cols else 'mean'
#               for col in df.columns if col != 'date'}
#

In [7]:
# Group By Continent

continent_df = df.groupby(['region', 'Year'], dropna=False).agg(nanmean).reset_index()
continent_df

,region,Year,longitude,latitude,Population growth (annual %),"Population, total","Current account balance, Percent of GDP,","Employment, Persons, Millions","General government gross debt, Percent of GDP,","General government net debt, National currency, Billions",...,"Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units","Gross domestic product per capita, current prices, Purchasing power parity; international dollars, Units","Gross domestic product per capita, current prices, U.S. dollars, Units","Gross domestic product, current prices, Purchasing power parity; international dollars, Billions","Gross domestic product, current prices, U.S. dollars, Billions","Gross national savings, Percent of GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,"
0,East Asia & Pacific,1980.0,102.080504,4.391124,1.947838,4.166191e+07,-2.671579,13.360000,48.810000,44619.100000,...,8329.468947,3367.007368,2761.922105,100.780476,92.535714,22.197857,78.182632,10.86,29.032667,4.545714
1,East Asia & Pacific,1981.0,102.080504,4.391124,2.281081,4.229082e+07,-3.943158,13.531667,53.950000,57895.000000,...,8642.021579,3825.336316,2935.958947,115.844762,99.090000,22.346429,79.358947,7.93,31.001333,4.290000
2,East Asia & Pacific,1982.0,102.080504,4.391124,2.442973,4.298174e+07,-3.261053,13.731667,58.960000,73262.900000,...,8758.651579,4118.301053,2894.603684,128.212857,95.854762,22.620000,80.647895,7.06,30.174000,4.431429
3,East Asia & Pacific,1983.0,102.080504,4.391124,2.454054,4.367263e+07,-2.458421,13.896667,64.920000,90702.100000,...,8940.959474,4374.371053,2920.687895,139.860000,102.288095,22.815000,81.811579,6.55,29.289333,5.030000
4,East Asia & Pacific,1984.0,102.080504,4.391124,2.448919,4.432978e+07,0.043158,13.988333,66.970000,51893.755000,...,9457.106842,4793.277895,3116.381053,154.646190,109.466667,23.585000,82.974211,6.46,28.070667,4.532500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,Sub-Saharan Africa,2021.0,16.672209,-0.974403,NaN,2.472264e+07,-8.553333,NaN,68.834255,5149.842778,...,4977.253617,5377.209574,2216.330000,93.687917,36.750833,15.881111,24.124681,NaN,24.480682,16.200000
319,Sub-Saharan Africa,2022.0,16.672209,-0.974403,NaN,2.535862e+07,-8.376667,NaN,66.907447,5895.420556,...,5074.848511,5580.184255,2368.989149,99.045208,40.133750,16.299333,24.737234,NaN,24.672273,15.178000
320,Sub-Saharan Africa,2023.0,16.672209,-0.974403,NaN,2.600404e+07,-7.247500,NaN,64.574468,6657.813333,...,5190.082766,5810.318511,2506.241915,105.166875,43.627083,17.566000,25.365319,NaN,24.637500,15.138000
321,Sub-Saharan Africa,2024.0,16.672209,-0.974403,NaN,2.665996e+07,-6.266667,NaN,62.263830,7493.871667,...,5297.861489,6044.392553,2637.748511,111.791458,47.574167,18.633111,26.007872,NaN,24.635000,15.072000


In [8]:
# Output DF Final Data

continent_df = continent_df.to_csv('../3_Outputs/Data/ba_continent_df.csv', index = False)

*Create East Asian, South Asian versus All Other Categories*

In [9]:
# Read CSV

continent_final_df = pd.read_csv('../3_Outputs/Data/ba_continent_df.csv')
continent_final_df

,region,Year,longitude,latitude,Population growth (annual %),"Population, total","Current account balance, Percent of GDP,","Employment, Persons, Millions","General government gross debt, Percent of GDP,","General government net debt, National currency, Billions",...,"Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units","Gross domestic product per capita, current prices, Purchasing power parity; international dollars, Units","Gross domestic product per capita, current prices, U.S. dollars, Units","Gross domestic product, current prices, Purchasing power parity; international dollars, Billions","Gross domestic product, current prices, U.S. dollars, Billions","Gross national savings, Percent of GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,"
0,East Asia & Pacific,1980.0,102.080504,4.391124,1.947838,4.166191e+07,-2.671579,13.360000,48.810000,44619.100000,...,8329.468947,3367.007368,2761.922105,100.780476,92.535714,22.197857,78.182632,10.86,29.032667,4.545714
1,East Asia & Pacific,1981.0,102.080504,4.391124,2.281081,4.229082e+07,-3.943158,13.531667,53.950000,57895.000000,...,8642.021579,3825.336316,2935.958947,115.844762,99.090000,22.346429,79.358947,7.93,31.001333,4.290000
2,East Asia & Pacific,1982.0,102.080504,4.391124,2.442973,4.298174e+07,-3.261053,13.731667,58.960000,73262.900000,...,8758.651579,4118.301053,2894.603684,128.212857,95.854762,22.620000,80.647895,7.06,30.174000,4.431429
3,East Asia & Pacific,1983.0,102.080504,4.391124,2.454054,4.367263e+07,-2.458421,13.896667,64.920000,90702.100000,...,8940.959474,4374.371053,2920.687895,139.860000,102.288095,22.815000,81.811579,6.55,29.289333,5.030000
4,East Asia & Pacific,1984.0,102.080504,4.391124,2.448919,4.432978e+07,0.043158,13.988333,66.970000,51893.755000,...,9457.106842,4793.277895,3116.381053,154.646190,109.466667,23.585000,82.974211,6.46,28.070667,4.532500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,Sub-Saharan Africa,2021.0,16.672209,-0.974403,NaN,2.472264e+07,-8.553333,NaN,68.834255,5149.842778,...,4977.253617,5377.209574,2216.330000,93.687917,36.750833,15.881111,24.124681,NaN,24.480682,16.200000
319,Sub-Saharan Africa,2022.0,16.672209,-0.974403,NaN,2.535862e+07,-8.376667,NaN,66.907447,5895.420556,...,5074.848511,5580.184255,2368.989149,99.045208,40.133750,16.299333,24.737234,NaN,24.672273,15.178000
320,Sub-Saharan Africa,2023.0,16.672209,-0.974403,NaN,2.600404e+07,-7.247500,NaN,64.574468,6657.813333,...,5190.082766,5810.318511,2506.241915,105.166875,43.627083,17.566000,25.365319,NaN,24.637500,15.138000
321,Sub-Saharan Africa,2024.0,16.672209,-0.974403,NaN,2.665996e+07,-6.266667,NaN,62.263830,7493.871667,...,5297.861489,6044.392553,2637.748511,111.791458,47.574167,18.633111,26.007872,NaN,24.635000,15.072000


In [10]:
# Create East Asian Filter

east_asian_df = continent_final_df[(continent_final_df.region =='East Asia & Pacific')].reset_index()
east_asian_df

,index,region,Year,longitude,latitude,Population growth (annual %),"Population, total","Current account balance, Percent of GDP,","Employment, Persons, Millions","General government gross debt, Percent of GDP,",...,"Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units","Gross domestic product per capita, current prices, Purchasing power parity; international dollars, Units","Gross domestic product per capita, current prices, U.S. dollars, Units","Gross domestic product, current prices, Purchasing power parity; international dollars, Billions","Gross domestic product, current prices, U.S. dollars, Billions","Gross national savings, Percent of GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,"
0,0,East Asia & Pacific,1980.0,102.080504,4.391124,1.947838,4.166191e+07,-2.671579,13.360000,48.810000,...,8329.468947,3367.007368,2761.922105,100.780476,92.535714,22.197857,78.182632,10.86,29.032667,4.545714
1,1,East Asia & Pacific,1981.0,102.080504,4.391124,2.281081,4.229082e+07,-3.943158,13.531667,53.950000,...,8642.021579,3825.336316,2935.958947,115.844762,99.090000,22.346429,79.358947,7.93,31.001333,4.290000
2,2,East Asia & Pacific,1982.0,102.080504,4.391124,2.442973,4.298174e+07,-3.261053,13.731667,58.960000,...,8758.651579,4118.301053,2894.603684,128.212857,95.854762,22.620000,80.647895,7.06,30.174000,4.431429
3,3,East Asia & Pacific,1983.0,102.080504,4.391124,2.454054,4.367263e+07,-2.458421,13.896667,64.920000,...,8940.959474,4374.371053,2920.687895,139.860000,102.288095,22.815000,81.811579,6.55,29.289333,5.030000
4,4,East Asia & Pacific,1984.0,102.080504,4.391124,2.448919,4.432978e+07,0.043158,13.988333,66.970000,...,9457.106842,4793.277895,3116.381053,154.646190,109.466667,23.585000,82.974211,6.46,28.070667,4.532500
5,5,East Asia & Pacific,1985.0,102.080504,4.391124,2.375135,4.500280e+07,2.552000,14.191667,44.633333,...,13206.594000,7524.953500,3828.044500,161.736818,105.175909,21.748571,80.019000,6.56,27.420667,4.856000
6,6,East Asia & Pacific,1986.0,102.080504,4.391124,2.333514,4.571834e+07,5.851500,14.411667,48.043333,...,13287.888500,7690.934000,3674.333333,173.450455,131.905652,20.429333,77.792381,5.02,25.725625,5.272727
7,7,East Asia & Pacific,1987.0,102.080504,4.391124,2.358378,4.647175e+07,5.825500,14.693333,46.746667,...,13036.342857,7704.348095,4305.981429,181.976957,157.373913,24.696667,79.080000,4.27,26.090625,5.432727
8,8,East Asia & Pacific,1988.0,102.080504,4.391124,2.285135,4.723217e+07,3.968000,14.998333,32.525000,...,13366.093333,8149.098571,4968.730476,203.551739,190.467391,24.471333,80.333333,4.56,26.403750,5.125455
9,9,East Asia & Pacific,1989.0,102.080504,4.391124,2.200270,4.797578e+07,2.754000,15.365000,28.632000,...,13763.540476,8705.560952,5287.992857,223.432174,196.754783,24.756000,81.571429,5.50,27.893125,4.568182


In [11]:
# Create South Asian Filter

south_asian_df = continent_final_df[(continent_final_df.region =='South Asia')].reset_index()
south_asian_df

,index,region,Year,longitude,latitude,Population growth (annual %),"Population, total","Current account balance, Percent of GDP,","Employment, Persons, Millions","General government gross debt, Percent of GDP,",...,"Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units","Gross domestic product per capita, current prices, Purchasing power parity; international dollars, Units","Gross domestic product per capita, current prices, U.S. dollars, Units","Gross domestic product, current prices, Purchasing power parity; international dollars, Billions","Gross domestic product, current prices, U.S. dollars, Billions","Gross national savings, Percent of GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,"
0,231,South Asia,1980.0,79.738687,22.968335,2.23875,1.125776e+08,-5.215000,NaN,NaN,...,2050.528571,774.410000,304.186667,72.281429,37.926667,12.375000,127.567143,NaN,24.020000,NaN
1,232,South Asia,1981.0,79.738687,22.968335,2.11500,1.152783e+08,-3.708571,NaN,NaN,...,2141.282857,884.652857,320.740000,83.875714,37.605714,14.141429,130.661429,NaN,19.953333,NaN
2,233,South Asia,1982.0,79.738687,22.968335,2.00625,1.180394e+08,-11.907143,NaN,NaN,...,2216.152857,971.835714,335.790000,92.698571,38.954286,14.304286,133.827143,NaN,21.945000,NaN
3,234,South Asia,1983.0,79.738687,22.968335,1.94125,1.208546e+08,-12.215714,NaN,NaN,...,2256.704286,1029.648571,341.257143,102.831429,41.222857,15.041429,137.048571,NaN,22.413333,3.850000
4,235,South Asia,1984.0,79.738687,22.968335,1.94625,1.237160e+08,-11.307143,NaN,NaN,...,2410.111429,1135.340000,367.682857,110.840000,41.281429,16.372857,140.310000,NaN,22.826667,3.750000
5,236,South Asia,1985.0,79.738687,22.968335,2.00500,1.266178e+08,-8.948571,NaN,NaN,...,2542.340000,1233.955714,383.194286,120.790000,44.681429,16.977143,143.601429,NaN,23.718333,3.650000
6,237,South Asia,1986.0,79.738687,22.968335,2.07375,1.295547e+08,-8.755714,NaN,NaN,...,2634.228571,1303.658571,408.801429,129.304286,47.197143,15.800000,146.922857,NaN,22.531667,3.300000
7,238,South Asia,1987.0,79.738687,22.968335,2.13375,1.325261e+08,-5.832857,NaN,NaN,...,2750.425714,1392.177143,419.087143,137.891429,52.300000,19.664286,150.270000,NaN,21.140000,3.070000
8,239,South Asia,1988.0,79.738687,22.968335,2.23875,1.355351e+08,-4.587143,NaN,NaN,...,2891.310000,1513.700000,462.027143,154.694286,55.950000,19.191429,153.637143,NaN,22.876667,3.140000
9,240,South Asia,1989.0,79.738687,22.968335,2.38625,1.385878e+08,-5.670000,NaN,NaN,...,3002.460000,1631.458571,475.195714,169.660000,56.875714,20.178571,157.025714,NaN,21.948333,3.140000


In [12]:
# Create Other Region DF

other_region_df = continent_final_df[(continent_final_df.region == 'North America') | 
                                         (continent_final_df.region == 'Europe & Central Asia') | 
                                         (continent_final_df.region == 'Latin America & Caribbean ') | 
                                         (continent_final_df.region == 'Middle East & North Africa') | 
                                         (continent_final_df.region == 'Sub-Saharan Africa ')].reset_index()
other_region_df

,index,region,Year,longitude,latitude,Population growth (annual %),"Population, total","Current account balance, Percent of GDP,","Employment, Persons, Millions","General government gross debt, Percent of GDP,",...,"Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units","Gross domestic product per capita, current prices, Purchasing power parity; international dollars, Units","Gross domestic product per capita, current prices, U.S. dollars, Units","Gross domestic product, current prices, Purchasing power parity; international dollars, Billions","Gross domestic product, current prices, U.S. dollars, Billions","Gross national savings, Percent of GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,"
0,47,Europe & Central Asia,1980.0,19.679874,48.059468,0.926491,1.378838e+07,-2.995000,8.847647,35.187500,...,24423.268400,9191.055600,9140.380800,172.291200,163.962400,24.236000,19.615600,NaN,26.866667,4.398000
1,48,Europe & Central Asia,1981.0,19.679874,48.059468,0.908070,1.387991e+07,-3.017500,8.309444,38.141250,...,24541.055200,10110.306800,8369.202800,188.921200,148.006000,23.140800,19.734800,NaN,26.615417,5.196000
2,49,Europe & Central Asia,1982.0,19.679874,48.059468,0.863333,1.396321e+07,-2.202917,8.248889,39.591111,...,24739.625600,10820.992000,8006.588800,202.171200,143.070800,23.217600,19.827600,NaN,26.330000,5.733333
3,50,Europe & Central Asia,1983.0,19.679874,48.059468,0.850000,1.404274e+07,-1.189167,8.197222,42.536667,...,25092.598000,11408.984400,7693.258000,214.510400,139.349600,23.018400,19.918800,NaN,25.360417,6.220000
4,51,Europe & Central Asia,1984.0,19.679874,48.059468,0.838772,1.412585e+07,-0.417917,8.216111,45.321111,...,25832.996800,12173.757600,7517.024800,228.225600,133.906800,23.705200,19.998000,NaN,25.747200,6.570476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,318,Sub-Saharan Africa,2021.0,16.672209,-0.974403,NaN,2.472264e+07,-8.553333,NaN,68.834255,...,4977.253617,5377.209574,2216.330000,93.687917,36.750833,15.881111,24.124681,NaN,24.480682,16.200000
226,319,Sub-Saharan Africa,2022.0,16.672209,-0.974403,NaN,2.535862e+07,-8.376667,NaN,66.907447,...,5074.848511,5580.184255,2368.989149,99.045208,40.133750,16.299333,24.737234,NaN,24.672273,15.178000
227,320,Sub-Saharan Africa,2023.0,16.672209,-0.974403,NaN,2.600404e+07,-7.247500,NaN,64.574468,...,5190.082766,5810.318511,2506.241915,105.166875,43.627083,17.566000,25.365319,NaN,24.637500,15.138000
228,321,Sub-Saharan Africa,2024.0,16.672209,-0.974403,NaN,2.665996e+07,-6.266667,NaN,62.263830,...,5297.861489,6044.392553,2637.748511,111.791458,47.574167,18.633111,26.007872,NaN,24.635000,15.072000


In [13]:
# Groupby

other_region_mean_df = other_region_df.groupby('Year').agg('mean')
other_region_mean_df.head()

,index,longitude,latitude,Population growth (annual %),"Population, total","Current account balance, Percent of GDP,","Employment, Persons, Millions","General government gross debt, Percent of GDP,","General government net debt, National currency, Billions","General government net debt, Percent of GDP,",...,"Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units","Gross domestic product per capita, current prices, Purchasing power parity; international dollars, Units","Gross domestic product per capita, current prices, U.S. dollars, Units","Gross domestic product, current prices, Purchasing power parity; international dollars, Billions","Gross domestic product, current prices, U.S. dollars, Billions","Gross national savings, Percent of GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,"
Year,,,,,,,,,,,,,,,,,,,,,
1980.0,148.2,-14.38039,24.556544,2.164992,2.474400e+07,-0.032327,16.501912,75.442500,44.188333,24.3500,...,22705.218696,9562.802220,6828.203464,375.287275,359.574088,20.734828,35.080583,14.01,26.185971,9.317206
1981.0,149.2,-14.38039,24.556544,2.053876,2.507833e+07,-2.055687,16.544861,101.870417,48.883333,24.0650,...,22100.151373,10154.632938,6923.892991,419.696430,395.369263,20.244515,35.456812,16.82,26.040649,9.395059
1982.0,150.2,-14.38039,24.556544,2.031500,2.541273e+07,-1.588744,16.374722,117.443704,56.128750,21.8275,...,20977.801912,10167.967120,6671.060167,437.865457,407.729754,20.572082,35.891168,13.59,25.090681,10.569728
1983.0,151.2,-14.38039,24.556544,2.005538,2.574279e+07,-2.054289,16.393056,119.812222,78.043000,23.7730,...,20688.984728,10367.357476,6632.528161,472.298249,438.326323,18.370745,36.317161,9.90,23.535454,11.082767
1984.0,152.2,-14.38039,24.556544,2.007844,2.606967e+07,-0.804848,16.967778,103.528704,95.200000,25.7950,...,21220.768087,11008.332790,6722.579656,520.290817,478.975383,19.583326,36.738767,11.19,23.624477,10.777629


In [14]:
# Create new variables

other_region_mean_df['region'] = 'Combined Other Five Regions'
other_region_mean_df['Year'] = other_region_mean_df.index
other_region_mean_df.head()

,index,longitude,latitude,Population growth (annual %),"Population, total","Current account balance, Percent of GDP,","Employment, Persons, Millions","General government gross debt, Percent of GDP,","General government net debt, National currency, Billions","General government net debt, Percent of GDP,",...,"Gross domestic product per capita, current prices, U.S. dollars, Units","Gross domestic product, current prices, Purchasing power parity; international dollars, Billions","Gross domestic product, current prices, U.S. dollars, Billions","Gross national savings, Percent of GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,",region,Year
Year,,,,,,,,,,,,,,,,,,,,,
1980.0,148.2,-14.38039,24.556544,2.164992,2.474400e+07,-0.032327,16.501912,75.442500,44.188333,24.3500,...,6828.203464,375.287275,359.574088,20.734828,35.080583,14.01,26.185971,9.317206,Combined Other Five Regions,1980.0
1981.0,149.2,-14.38039,24.556544,2.053876,2.507833e+07,-2.055687,16.544861,101.870417,48.883333,24.0650,...,6923.892991,419.696430,395.369263,20.244515,35.456812,16.82,26.040649,9.395059,Combined Other Five Regions,1981.0
1982.0,150.2,-14.38039,24.556544,2.031500,2.541273e+07,-1.588744,16.374722,117.443704,56.128750,21.8275,...,6671.060167,437.865457,407.729754,20.572082,35.891168,13.59,25.090681,10.569728,Combined Other Five Regions,1982.0
1983.0,151.2,-14.38039,24.556544,2.005538,2.574279e+07,-2.054289,16.393056,119.812222,78.043000,23.7730,...,6632.528161,472.298249,438.326323,18.370745,36.317161,9.90,23.535454,11.082767,Combined Other Five Regions,1983.0
1984.0,152.2,-14.38039,24.556544,2.007844,2.606967e+07,-0.804848,16.967778,103.528704,95.200000,25.7950,...,6722.579656,520.290817,478.975383,19.583326,36.738767,11.19,23.624477,10.777629,Combined Other Five Regions,1984.0


In [15]:
# Concat Three DFs Together

frames = [east_asian_df, south_asian_df, other_region_mean_df]

continental_graph_df = pd.concat(frames, ignore_index = True)
continental_graph_df = continental_graph_df.drop(['index'], axis=1)
continental_graph_df

,region,Year,longitude,latitude,Population growth (annual %),"Population, total","Current account balance, Percent of GDP,","Employment, Persons, Millions","General government gross debt, Percent of GDP,","General government net debt, National currency, Billions",...,"Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units","Gross domestic product per capita, current prices, Purchasing power parity; international dollars, Units","Gross domestic product per capita, current prices, U.S. dollars, Units","Gross domestic product, current prices, Purchasing power parity; international dollars, Billions","Gross domestic product, current prices, U.S. dollars, Billions","Gross national savings, Percent of GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,"
0,East Asia & Pacific,1980.0,102.080504,4.391124,1.947838,4.166191e+07,-2.671579,13.360000,48.810000,44619.100000,...,8329.468947,3367.007368,2761.922105,100.780476,92.535714,22.197857,78.182632,10.86,29.032667,4.545714
1,East Asia & Pacific,1981.0,102.080504,4.391124,2.281081,4.229082e+07,-3.943158,13.531667,53.950000,57895.000000,...,8642.021579,3825.336316,2935.958947,115.844762,99.090000,22.346429,79.358947,7.93,31.001333,4.290000
2,East Asia & Pacific,1982.0,102.080504,4.391124,2.442973,4.298174e+07,-3.261053,13.731667,58.960000,73262.900000,...,8758.651579,4118.301053,2894.603684,128.212857,95.854762,22.620000,80.647895,7.06,30.174000,4.431429
3,East Asia & Pacific,1983.0,102.080504,4.391124,2.454054,4.367263e+07,-2.458421,13.896667,64.920000,90702.100000,...,8940.959474,4374.371053,2920.687895,139.860000,102.288095,22.815000,81.811579,6.55,29.289333,5.030000
4,East Asia & Pacific,1984.0,102.080504,4.391124,2.448919,4.432978e+07,0.043158,13.988333,66.970000,51893.755000,...,9457.106842,4793.277895,3116.381053,154.646190,109.466667,23.585000,82.974211,6.46,28.070667,4.532500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Combined Other Five Regions,2021.0,-14.380390,24.556544,NaN,5.300439e+07,-4.851083,23.916950,80.462533,338642.413089,...,27098.533742,29346.097972,22671.717742,2688.168124,2540.162328,18.291812,54.316246,0.41,23.224810,10.306273
135,Combined Other Five Regions,2022.0,-14.380390,24.556544,NaN,5.345879e+07,-4.076186,NaN,79.868244,417685.624486,...,27879.224928,30731.603981,23831.433212,2818.627842,2666.489863,19.212038,54.752162,NaN,23.323842,9.465658
136,Combined Other Five Regions,2023.0,-14.380390,24.556544,NaN,5.391120e+07,-3.422181,NaN,78.958078,522116.235670,...,28515.436616,31923.148144,24901.097038,2936.978821,2781.695211,19.981899,55.149817,NaN,23.358936,9.168451
137,Combined Other Five Regions,2024.0,-14.380390,24.556544,NaN,5.436135e+07,-2.997837,NaN,77.979511,661198.283142,...,28949.861097,33029.238108,25852.261284,3052.211093,2893.316342,20.610203,55.598890,NaN,23.517637,9.016134


In [16]:
# Create Growth Rates

continental_graph_df['GDP per capita growth rate'] = continental_graph_df['Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units'].pct_change()
continental_graph_df['Investment growth rate'] = continental_graph_df['Total investment, Percent of GDP, '].pct_change()
continental_graph_df

,region,Year,longitude,latitude,Population growth (annual %),"Population, total","Current account balance, Percent of GDP,","Employment, Persons, Millions","General government gross debt, Percent of GDP,","General government net debt, National currency, Billions",...,"Gross domestic product per capita, current prices, U.S. dollars, Units","Gross domestic product, current prices, Purchasing power parity; international dollars, Billions","Gross domestic product, current prices, U.S. dollars, Billions","Gross national savings, Percent of GDP,","Population, Persons, Millions","Six-month London interbank offered rate (LIBOR), Percent,","Total investment, Percent of GDP,","Unemployment rate, Percent of total labor force,",GDP per capita growth rate,Investment growth rate
0,East Asia & Pacific,1980.0,102.080504,4.391124,1.947838,4.166191e+07,-2.671579,13.360000,48.810000,44619.100000,...,2761.922105,100.780476,92.535714,22.197857,78.182632,10.86,29.032667,4.545714,NaN,NaN
1,East Asia & Pacific,1981.0,102.080504,4.391124,2.281081,4.229082e+07,-3.943158,13.531667,53.950000,57895.000000,...,2935.958947,115.844762,99.090000,22.346429,79.358947,7.93,31.001333,4.290000,0.037524,0.067809
2,East Asia & Pacific,1982.0,102.080504,4.391124,2.442973,4.298174e+07,-3.261053,13.731667,58.960000,73262.900000,...,2894.603684,128.212857,95.854762,22.620000,80.647895,7.06,30.174000,4.431429,0.013496,-0.026687
3,East Asia & Pacific,1983.0,102.080504,4.391124,2.454054,4.367263e+07,-2.458421,13.896667,64.920000,90702.100000,...,2920.687895,139.860000,102.288095,22.815000,81.811579,6.55,29.289333,5.030000,0.020815,-0.029319
4,East Asia & Pacific,1984.0,102.080504,4.391124,2.448919,4.432978e+07,0.043158,13.988333,66.970000,51893.755000,...,3116.381053,154.646190,109.466667,23.585000,82.974211,6.46,28.070667,4.532500,0.057728,-0.041608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Combined Other Five Regions,2021.0,-14.380390,24.556544,NaN,5.300439e+07,-4.851083,23.916950,80.462533,338642.413089,...,22671.717742,2688.168124,2540.162328,18.291812,54.316246,0.41,23.224810,10.306273,0.037875,0.020326
135,Combined Other Five Regions,2022.0,-14.380390,24.556544,NaN,5.345879e+07,-4.076186,NaN,79.868244,417685.624486,...,23831.433212,2818.627842,2666.489863,19.212038,54.752162,NaN,23.323842,9.465658,0.028809,0.004264
136,Combined Other Five Regions,2023.0,-14.380390,24.556544,NaN,5.391120e+07,-3.422181,NaN,78.958078,522116.235670,...,24901.097038,2936.978821,2781.695211,19.981899,55.149817,NaN,23.358936,9.168451,0.022820,0.001505
137,Combined Other Five Regions,2024.0,-14.380390,24.556544,NaN,5.436135e+07,-2.997837,NaN,77.979511,661198.283142,...,25852.261284,3052.211093,2893.316342,20.610203,55.598890,NaN,23.517637,9.016134,0.015235,0.006794


In [86]:
# Output DF Final Data

continental_graph_df = continental_graph_df.to_csv('../3_Outputs/Data/continental_df.csv', index = False)

In [96]:
# Read in Continental_final_df

continental_graph_df = pd.read_csv('../3_Outputs/Data/continental_final_df.csv')

*Melts*

In [97]:
# Create years list

vars_list = ["Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units", 
             "Population, total", 
             'Total investment, Percent of GDP, ']
vars_list

['Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units',
 'Population, total',
 'Total investment, Percent of GDP, ']

In [98]:
# Create years list 2

vars_list2 = ["GDP per capita growth rate", 
             "Population growth (annual %)", 
             'Investment growth rate']
vars_list2

['GDP per capita growth rate',
 'Population growth (annual %)',
 'Investment growth rate']

In [99]:
# Melt

continental_graph_df2 = pd.melt(continental_graph_df, id_vars=['region', 'Year'], 
                 value_vars=vars_list)
continental_graph_df2

,region,Year,variable,value
0,East Asia & Pacific,1980.0,"Gross domestic product per capita, constant pr...",8329.468947
1,East Asia & Pacific,1981.0,"Gross domestic product per capita, constant pr...",8642.021579
2,East Asia & Pacific,1982.0,"Gross domestic product per capita, constant pr...",8758.651579
3,East Asia & Pacific,1983.0,"Gross domestic product per capita, constant pr...",8940.959474
4,East Asia & Pacific,1984.0,"Gross domestic product per capita, constant pr...",9457.106842
...,...,...,...,...
412,Combined Other Five Regions,2021.0,"Total investment, Percent of GDP,",23.224810
413,Combined Other Five Regions,2022.0,"Total investment, Percent of GDP,",23.323842
414,Combined Other Five Regions,2023.0,"Total investment, Percent of GDP,",23.358936
415,Combined Other Five Regions,2024.0,"Total investment, Percent of GDP,",23.517637


In [100]:
# Melt 2

continental_graph_df3 = pd.melt(continental_graph_df, id_vars=['region', 'Year'], 
                 value_vars=vars_list2)
continental_graph_df3

,region,Year,variable,value
0,East Asia & Pacific,1980.0,GDP per capita growth rate,NaN
1,East Asia & Pacific,1981.0,GDP per capita growth rate,0.037524
2,East Asia & Pacific,1982.0,GDP per capita growth rate,0.013496
3,East Asia & Pacific,1983.0,GDP per capita growth rate,0.020815
4,East Asia & Pacific,1984.0,GDP per capita growth rate,0.057728
...,...,...,...,...
412,Combined Other Five Regions,2021.0,Investment growth rate,0.020326
413,Combined Other Five Regions,2022.0,Investment growth rate,0.004264
414,Combined Other Five Regions,2023.0,Investment growth rate,0.001505
415,Combined Other Five Regions,2024.0,Investment growth rate,0.006794


In [101]:
# Replace Variable Names

continental_graph_df2['variable'] = continental_graph_df2['variable'].replace('Population, total', 'Total Population')
continental_graph_df2['variable'] = continental_graph_df2['variable'].replace('Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units', 'GDP per capita')
continental_graph_df2['variable'] = continental_graph_df2['variable'].replace('Total investment, Percent of GDP, ', 'Total Investment')
continental_graph_df2

,region,Year,variable,value
0,East Asia & Pacific,1980.0,GDP per capita,8329.468947
1,East Asia & Pacific,1981.0,GDP per capita,8642.021579
2,East Asia & Pacific,1982.0,GDP per capita,8758.651579
3,East Asia & Pacific,1983.0,GDP per capita,8940.959474
4,East Asia & Pacific,1984.0,GDP per capita,9457.106842
...,...,...,...,...
412,Combined Other Five Regions,2021.0,Total Investment,23.224810
413,Combined Other Five Regions,2022.0,Total Investment,23.323842
414,Combined Other Five Regions,2023.0,Total Investment,23.358936
415,Combined Other Five Regions,2024.0,Total Investment,23.517637


----

**Create Graphs**

*Facet Plotly Graph 3 x 1 for GDP per capita, Population & Investment*

In [102]:
# Figure 1
# https://plotly.com/python/facet-plots/

fig = px.line(continental_graph_df2, x="Year", y="value", facet_col="variable", color = 'region',
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              facet_col_spacing=0.1, # default is 0.03
              #height=600, width=800,
              title="Strong Economic, Population & Investment Growth in East & South Asian Markets" 
              #category_orders={"variable": ["GDP per capita", "Total Population", "Total Investment"]} 
             )

# Remove Facet Title Prompt
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


# Independent Y-Axes
fig.update_yaxes(matches=None)
fig.update_yaxes(title_text="<b>GDP per capita, thous. PPP 2017$</b>", showticklabels=True, col=1) # assuming second facet
fig.update_yaxes(title_text="<b>Population (millions)</b>", showticklabels=True, col=2) # assuming second facet
fig.update_yaxes(title_text="<b>Investment, % of GDP</b>", showticklabels=True, col=3) # assuming second facet
#fig.update_yaxes(title_text="<b>%</b> of Global GDP", secondary_y=False)
#fig.update_yaxes(title_text="<b>%</b> of Global GDP", secondary_y=True)

# Add bolding lines
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

# Move legend
fig.update_layout(
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.35,
    xanchor="right",
    x=0.9
))
fig.update_layout(legend_title_text='')

# Output
fig.show()
fig.write_image("../3_Outputs/Figures/East & South Asian Growth 1.jpg")
pio.write_html(fig, file="../3_Outputs/Figures/East & South Asian Growth 1.html", auto_open=True)

## Output to Datapane API
#report = dp.Report(dp.Plot(fig) ) #Create a report
#report.publish(name='fdiplot1', open=True, visibility='PUBLIC') #Publish the report

In [103]:
# Figure 1b: Forecast
# https://plotly.com/python/facet-plots/

fig = px.line(continental_graph_df2, x="Year", y="value", facet_col="variable", color = 'region',
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              facet_col_spacing=0.1, # default is 0.03
              #height=600, width=800,
              range_x=['1980', '2025'],
              title="Strong Economic, Population & Investment Growth in East & South Asian Markets" 
              #category_orders={"variable": ["GDP per capita", "Total Population", "Total Investment"]} 
             )

# Remove Facet Title Prompt
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


# Independent Y-Axes
fig.update_yaxes(matches=None)
fig.update_yaxes(title_text="<b>GDP per capita, thous. PPP 2017$</b>", showticklabels=True, title_standoff = 0.1, title_font = dict(size=10), col=1) # assuming second facet
fig.update_yaxes(title_text="<b>Population (millions)</b>", showticklabels=True, title_standoff = 0.1, title_font = dict(size=10), col=2) # assuming second facet
fig.update_yaxes(title_text="<b>Investment, % of GDP</b>", showticklabels=True, title_standoff = 0.1, title_font = dict(size=10), col=3) # assuming second facet
#fig.update_yaxes(title_text="<b>%</b> of Global GDP", secondary_y=False)
#fig.update_yaxes(title_text="<b>%</b> of Global GDP", secondary_y=True)

# Change X-Axis
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1980,
        dtick = 5
    ),
    xaxis2 = dict(
        tickmode = 'linear',
        tick0 = 1980,
        dtick = 5
    ),
     xaxis3 = dict(
        tickmode = 'linear',
        tick0 = 1980,
        dtick = 5
    )
)

# Add Vertical Line

fig.update_layout(shapes=[
    dict(
      type= 'line',
      yref= 'y', y0= 0, y1= 35000,
      xref= 'x', x0= 2019, x1= 2019,
        line=dict(
                 #color="LightSeaGreen",
                 width=1.5,
                 dash="dash")),
    dict(
      type= 'line',
      yref= 'y2', y0= 0, y1= 260000000,
      xref= 'x2', x0= 2019, x1= 2019,
        line=dict(
                 #color="LightSeaGreen",
                 width=1.5,
                 dash="dash")),
    dict(
      type= 'line',
      yref= 'y3', y0= 0, y1= 36,
      xref= 'x3', x0= 2019, x1= 2019,
        line=dict(
                 #color="LightSeaGreen",
                 width=1.5,
                 dash="dash"))
])

# Add bolding lines
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

# Move legend
fig.update_layout(
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.35,
    xanchor="right",
    x=0.9
))
fig.update_layout(legend_title_text='')

# Add Note About Forecast

fig.update_layout(annotations=[
       go.layout.Annotation(
            showarrow=False,
            text='Right of Dashed Lines Represent IMF & World Bank Forecast Data',
            xanchor='left',
            x=-0.085,
            #xshift=0,
            yanchor='top',
            y=1.12,
            font=dict(
                #family="Courier New, monospace",
                size=12,
                color="#696969"
            )
        )])

# Output
fig.show()
fig.write_image("../3_Outputs/Figures/East & South Asian Growth Forecast 1b.jpg")
pio.write_html(fig, file="../3_Outputs/Figures/East & South Asian Growth Forecast 1b.html", auto_open=True)

## Output to Datapane API
#report = dp.Report(dp.Plot(fig) ) #Create a report
#report.publish(name='fdiplot1', open=True, visibility='PUBLIC') #Publish the report

In [61]:
# Figure 2
# https://plotly.com/python/facet-plots/

fig = px.line(continental_graph_df3, x="Year", y="value", facet_col="variable", color = 'region',
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              facet_col_spacing=0.10, # default is 0.03
              #height=600, width=800,
              title="Strong Economic, Population & Investment Growth in East & South Asian Markets" 
              #category_orders={"variable": ["GDP per capita", "Total Population", "Total Investment"]} 
             )

# Remove Facet Title Prompt
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


# Independent Y-Axes
fig.update_yaxes(matches=None)
fig.update_yaxes(title_text="<b>%</b>", showticklabels=True, col=1) # assuming second facet
#fig.update_yaxes(title_text="<b>Population (millions)</b>", showticklabels=True, col=2) # assuming second facet
#fig.update_yaxes(title_text="<b>Investment, % of GDP</b>", showticklabels=True, col=3) # assuming second facet

# Add bolding lines
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

# Move legend
fig.update_layout(
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.35,
    xanchor="right",
    x=0.9
))
fig.update_layout(legend_title_text='')

# Output
fig.show()
fig.write_image("../3_Outputs/Figures/East & South Asian Growth 2.jpg")
pio.write_html(fig, file="../3_Outputs/Figures/East & South Asian Growth 2.html", auto_open=True)

## Output to Datapane API
#report = dp.Report(dp.Plot(fig) ) #Create a report
#report.publish(name='fdiplot1', open=True, visibility='PUBLIC') #Publish the report

In [24]:
## Figure X
## Source: https://plotly.com/python/multiple-axes/
#
## Create 2 figures with secondary y-axis
#fig = make_subplots(rows=1, cols=3)
##fig = make_subplots(specs=[[{"secondary_y": True}]])
#
## Add traces for Fig1
#fig.append_trace(
#    go.Scatter(x=continental_graph_df["Year"], 
#               y=continental_graph_df["Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units"][(continental_graph_df.region == 'Combined Other Five Regions')],
#               #color='region',
#               name="GDP per capita, 2017 PPP"), row=1, col=1
#)
#
#fig.append_trace(
#    go.Scatter(x=continental_graph_df["Year"], 
#               y=continental_graph_df["Gross domestic product per capita, constant prices, Purchasing power parity; 2017 international dollar, Units"][(continental_graph_df.region == 'East Asia & Pacific')],
#               #color='region',
#               name="GDP per capita, 2017 PPP"), row=1, col=1
#)
#
## Add traces for Fig2
#fig.append_trace(
#    go.Scatter(x=continental_graph_df["Year"], 
#               y=continental_graph_df["Population, total"],
#               #color='region',
#               name="Total Population"), row=1, col=2
#)
#
#
## Add traces for Fig3
#fig.append_trace(
#    go.Scatter(x=continental_graph_df["Year"], 
#               y=continental_graph_df["Total investment, Percent of GDP, "],
#               #color='region',
#               name="Total Investment, Percent of GDP"), row=1, col=3
#)
#
## Add bolding lines
#fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
#fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
#
## Add figure title
#fig.update_layout(
#    title_text="Strong Economic, Population & Investment Growth in East & South Asian Markets"
#)
#
## Move legend
#fig.update_layout(
#    legend=dict(
#    orientation="h",
#    yanchor="bottom",
#    y=-0.35,
#    xanchor="right",
#    x=0.9
#))
#
### Add annotation
##fig.update_layout(
##    annotations=[
##        dict(
##            x=0.9,
##            y=-0.5,
##            text="Figure: Jake Schneider & Jeffry Frieden \nSource: World Bank World Development Indicators \nCode: https://github.com/jschneids13/COVID-Frieden-Schneider-Article",
##            showarrow=False,
##            xref="paper",
##            yref="paper"
##        )
#    ]
#)

## Set x-axis title
#fig.update_xaxes(title_text="Year")
#
### Set y-axes titles
##fig.update_yaxes(title_text="<b>%</b> of Global GDP", secondary_y=False)
##fig.update_yaxes(title_text="<b>%</b> of Global GDP", secondary_y=True)
#
## Output
#fig.show()
#fig.write_image("../3_Outputs/Figures/East & South Asian Growth 1.jpg")
#pio.write_html(fig, file="../3_Outputs/Figures/East & South Asian Growth 1.html", auto_open=True)
#
### Output to Datapane API
##report = dp.Report(dp.Plot(fig) ) #Create a report
##report.publish(name='fdiplot1', open=True, visibility='PUBLIC') #Publish the report